In [ ]:
from sklearn.preprocessing import LabelBinarizer as LB
from sklearn.preprocessing import normalize 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# --------------------
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.core import Reshape, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras import metrics
# --------------------
from pandas import DataFrame as df
# --------------------
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
# --------------------
import tarfile
import pickle
import random
import keras
import sys
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
directory=[]
for name in glob.glob('/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /0-5/*'): 
    directory.append(name) 
for name in glob.glob('/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /6-10/*'): 
    directory.append(name) 
for name in glob.glob('/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /11-15/*'): 
    directory.append(name) 
for name in glob.glob('/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /16-20/*'): 
    directory.append(name) 
for name in glob.glob('/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /21-25/*'): 
    directory.append(name) 
for name in glob.glob('/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /26-30/*'): 
    directory.append(name) 


In [ ]:
len(directory)

30

In [ ]:
directory

['/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /0-5/2.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /0-5/3.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /0-5/4.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /0-5/5.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /0-5/1.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /6-10/1.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /6-10/2.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /6-10/3.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /6-10/4.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /6-10/5.txt',
 '/content/drive/MyDrive/FYP - Neural 5G/Data /AWGN - Data (3000 samples) /11-15/1.txt',
 '/content/drive/MyDrive/FYP - Neura

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import asarray
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
import glob
from PIL import ImageFilter
import shutil
import glob
import shutil
from pylab import *
import matplotlib.gridspec as gridspec
import pathlib
from pathlib import Path

In [ ]:
master_data=[]
for i in range(len(directory)):
  targets=directory[i] 
  target_pss=np.loadtxt(targets, delimiter=",")
  target_pss=transpose(target_pss)
  target_pss=np.asarray(target_pss)
  master_data.append(target_pss)


In [ ]:
x=master_data[15]
x.shape

(2, 54800)

In [ ]:
master_data_2=[]
master_labels=np.zeros(shape=(3000,6))
master1=np.zeros(shape=(3000,2,548))
list_label=["0-5","6-10","11-15","16-20","21-25","26-30"]

In [ ]:
for j in range(30):
  for i in range(100):
    s=(100*j)+i
    master1[s,:,:]=master_data[j][:,(i*548):(548+(i*548))]
    


In [ ]:
final_data=master1
final_data.shape

(3000, 2, 548)

In [ ]:
for i in range(500):
  master_labels[i,0]=1
  master_labels[i+500,1]=1
  master_labels[i+1000,2]=1
  master_labels[i+1500,3]=1
  master_labels[i+2000,4]=1
  master_labels[i+2500,5]=1

master_labels.shape

(3000, 6)

In [ ]:
master_labels[499]

array([1., 0., 0., 0., 0., 0.])

In [ ]:
a=final_data
b=master_labels

In [ ]:
from mlxtend.preprocessing import shuffle_arrays_unison
X,Y=shuffle_arrays_unison(arrays=[a, b], random_seed=3)

In [ ]:
X_train=X[0:2500,:,:]
X_test=X[2500:,:,:]
Y_train=Y[0:2500,:]
Y_test=Y[2500:,:]

In [ ]:
in_shape = final_data[0].shape
out_shape = tuple([1]) + in_shape
print(in_shape)
print(out_shape)

(2, 548)
(1, 2, 548)


In [ ]:
dr = 0.5
model = Sequential()
model.add(Reshape(out_shape, input_shape = in_shape))
model.add(ZeroPadding2D((0, 2), data_format = 'channels_first'))
model.add(Conv2D(256, (1, 3), padding = 'valid', activation = "relu", name="conv1", kernel_initializer='glorot_uniform', data_format="channels_first"))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0,2), data_format = 'channels_first'))
model.add(Conv2D(80, (2, 3), activation="relu", name="conv3", padding="valid", kernel_initializer="glorot_uniform", data_format="channels_first"))
model.add(Dropout(dr))
model.add(Flatten())   
model.add(Dense(256, activation="relu", name="dense1", kernel_initializer="he_normal",))
model.add(Dropout(dr))
model.add(Dense(6, name="dense3", kernel_initializer="he_normal", activation = 'softmax'))
model.add(Reshape([len(list_label)]))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1, 2, 548)         0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 1, 2, 552)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 2, 550)       1024      
_________________________________________________________________
dropout (Dropout)            (None, 256, 2, 550)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 256, 2, 554)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 80, 1, 552)        122960    
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 1, 552)        0

In [ ]:
%load_ext tensorboard
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(X_train, Y_train, epochs = 50, batch_size = 64, validation_data=(X_test,Y_test), callbacks=[tensorboard_callback])

Epoch 1/50
40/40 [==============================] - 1s 33ms/step - loss: 0.2290 - accuracy: 0.9072 - val_loss: 0.7270 - val_accuracy: 0.8380
Epoch 2/50
40/40 [==============================] - 1s 27ms/step - loss: 0.2422 - accuracy: 0.9052 - val_loss: 0.6755 - val_accuracy: 0.8260
Epoch 3/50
40/40 [==============================] - 1s 27ms/step - loss: 0.1949 - accuracy: 0.9184 - val_loss: 0.8493 - val_accuracy: 0.8160
Epoch 4/50
40/40 [==============================] - 1s 27ms/step - loss: 0.2083 - accuracy: 0.9172 - val_loss: 0.7955 - val_accuracy: 0.8240
Epoch 5/50
40/40 [==============================] - 1s 27ms/step - loss: 0.2209 - accuracy: 0.9104 - val_loss: 0.6937 - val_accuracy: 0.8240
Epoch 6/50
40/40 [==============================] - 1s 27ms/step - loss: 0.2039 - accuracy: 0.9104 - val_loss: 0.6835 - val_accuracy: 0.8220
Epoch 7/50
40/40 [==============================] - 1s 27ms/step - loss: 0.1919 - accuracy: 0.9204 - val_loss: 0.7364 - val_accuracy: 0.8200
Epoch 8/50
40